In [3]:
import torch
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import csv
import random

from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from datetime import datetime

seed = 265
torch.manual_seed(seed)

device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cpu.


In [4]:
TOKENIZER = get_tokenizer('basic_english')
PATH_GENERATED = './generated/'
MIN_FREQ = 100

def read_files(datapath='./data_train/'):
    files = os.listdir(datapath)
    files = [datapath + f for f in files if f.endswith('.txt')]

    texts = []
    for file in files:
        with open(file) as f:
            texts += f.readlines()
    return texts

def tokenize(texts, tokenizer=TOKENIZER):
    tokenized_text = []
    for text in texts:
        tokenized_text += tokenizer(text)
    return tokenized_text

def yield_tokens(texts, tokenizer=TOKENIZER):
    """
    Remove yield tokens from the text before tokenizing
    """

    # Remove words with digits, upper case, and multiple space 
    no_digits = '\w*[0-9]+\w*'
    no_names = '\w*[A-Z]+\w*'
    no_spaces = '\s+'

    for text in texts:
        text = re.sub(no_digits, ' ', text)
        text = re.sub(no_names, ' ', text)
        text = re.sub(no_spaces, ' ', text)
        yield tokenizer(text)

def count_freqs(words, vocab):
    freqs = torch.zeros(len(vocab), dtype=torch.int)
    for w in words:
        freqs[vocab[w]] += 1
    return freqs

def create_vocabulary(lines, min_freq=MIN_FREQ):
    """
    Create a vocabulary (list of known tokens) from a list of strings
    """
    vocab = build_vocab_from_iterator(yield_tokens(lines), min_freq=min_freq, specials=["<unk>"])
    vocab.append_token("i")  # Upper case words like 'I' were removed so we should add it back again.
    vocab.set_default_index(vocab["<unk>"])
    return vocab

def calculate_word_weights(freqs):
    """
    Calculate the weight of each word so that the loss function can weigh 
    frequent words less and unfrequent words more.
    """
    total_words = sum(freqs)
    word_weights = [total_words / (len(freqs)* freq) for freq in freqs]
    word_weights = torch.tensor(word_weights, dtype=torch.float).to(device=device)
    return word_weights


In [5]:
# ----------------------- Tokenize texts -------------------------------

if os.path.isfile(PATH_GENERATED + "words_train.pt"):
    words_train = torch.load(PATH_GENERATED + "words_train.pt", map_location=torch.device(device))
    words_val   = torch.load(PATH_GENERATED + "words_val.pt", map_location=torch.device(device))
    words_test  = torch.load(PATH_GENERATED + "words_test.pt", map_location=torch.device(device))
        
else:
    lines_books_train = read_files('./data_train/')
    lines_books_val   = read_files('./data_val/')
    lines_books_test  = read_files('./data_test/')

    words_train = tokenize(lines_books_train)
    words_val   = tokenize(lines_books_val)
    words_test  = tokenize(lines_books_test)
    
    torch.save(words_train, PATH_GENERATED + "words_train.pt")
    torch.save(words_val, PATH_GENERATED + "words_val.pt")
    torch.save(words_test, PATH_GENERATED + "words_test.pt")



# ----------------------- Create vocabulary ----------------------------

VOCAB_FNAME = "vocabulary.pt"
if os.path.isfile(PATH_GENERATED + VOCAB_FNAME):
    vocab = torch.load(PATH_GENERATED + VOCAB_FNAME, map_location=torch.device(device))
else:
    vocab = create_vocabulary(lines_books_train, min_freq=MIN_FREQ)
    torch.save(vocab, PATH_GENERATED + VOCAB_FNAME)
    


# ------------------------ Quick analysis ------------------------------

VOCAB_SIZE = len(vocab)
freqs = count_freqs(words_train, vocab)
occurences = [(f.item(), w) for (f, w) in zip(freqs, vocab.lookup_tokens(range(VOCAB_SIZE)))]
word_weigts = calculate_word_weights(freqs)


In [6]:
n_print = 10
print("Total number of words in the training dataset:     ", len(words_train))
print("Total number of words in the validation dataset:   ", len(words_val))
print("Total number of words in the test dataset:         ", len(words_test))
print("Number of distinct words in the training dataset:  ", len(set(words_train)))
print("Number of distinct words kept (vocabulary size):   ", VOCAB_SIZE)

print(f"The {n_print} most occuring words:\n {occurences[:n_print]}")

Total number of words in the training dataset:      2684706
Total number of words in the validation dataset:    49526
Total number of words in the test dataset:          124152
Number of distinct words in the training dataset:   52105
Number of distinct words kept (vocabulary size):    1880
The 10 most occuring words:
 [(433907, '<unk>'), (182537, ','), (151278, 'the'), (123727, '.'), (82289, 'and'), (65661, 'of'), (62763, 'to'), (49230, 'a'), (41477, 'in'), (31052, 'that')]


In [7]:
CONTEXT_SIZE = 3

# ---------------- Define context / target pairs -----------------------
def compute_label(w):
    """
    helper function to define MAP_TARGET
    
    - 0 = 'unknown word'
    - 1 = 'punctuation' (i.e. the '<unk>' token)
    - 2 = 'is an actual word'
    """
    if w in ['<unk>']:
        return 0
    elif w in [',', '.', '(', ')', '?', '!']:
        return 1
    else:
        return 2

MAP_TARGET = {vocab[w]:compute_label(w) for w in vocab.lookup_tokens(range(VOCAB_SIZE))}

def create_dataset(text, vocab, context_size=CONTEXT_SIZE, map_target=MAP_TARGET):
    """
    Create a pytorch dataset of context / target pairs from a text
    """
    
    # Transform each word to its index in the vocabulary.
    txt = [vocab[w] for w in text]

    n_text = len(text)
    contexts = []
    targets = []
    for i in range(n_text - context_size):
        
        t = txt[i + context_size]
        if map_target[t] < 2: continue # We only want to guess actual words.
        c = txt[i:i + context_size]
        
        targets.append(t) 
        contexts.append(torch.tensor(c).to(device=device))
            
    contexts = torch.stack(contexts)
    targets = torch.tensor(targets).to(device=device)
    return TensorDataset(contexts, targets)

In [62]:
def load_dataset(words, vocab, fname):
    """
    Load dataset if already generated, otherwise, create it and save it
    """
    
    if os.path.isfile(PATH_GENERATED + fname):
        dataset = torch.load(PATH_GENERATED + fname, map_location=torch.device(device))
    else:
        dataset = create_dataset(words, vocab)
        torch.save(dataset, PATH_GENERATED + fname)
    return dataset

data_train = load_dataset(words_train, vocab, "data_train.pt")
data_val   = load_dataset(words_val, vocab, "data_val.pt")
data_test  = load_dataset(words_test, vocab, "data_test.pt")

In [9]:
class Word2Vec(nn.Module):
    
    def __init__(self, embedding, context_size=CONTEXT_SIZE):
        super().__init__()
        
        (vocab_size, embedding_dim) = embedding.weight.shape
        self.embedding = embedding

        self.fc1 = nn.Linear(embedding_dim*context_size, 128)
        self.fc2 = nn.Linear(128, vocab_size)

    def forward(self, x):
        out = self.embedding(x)
        out = F.relu(self.fc1(torch.flatten(out, 1)))        
        out = self.fc2(out)
        return out

In [10]:
def train(n_epochs, optimizer, model, loss_fn, train_loader, device=None):

    n_batch = len(train_loader)
    losses_train = []
    model.train()
    optimizer.zero_grad(set_to_none=True)

    for epoch in range(1, n_epochs + 1):

        loss_train = 0.0
        for contexts, targets in train_loader:

            contexts = contexts.to(device=device)
            targets = targets.to(device=device)

            outputs = model(contexts)

            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_train += loss.item()

        losses_train.append(loss_train / n_batch)

        if epoch == 1 or epoch % 5 == 0:
            print('{}  |  Epoch {}  |  Training loss {:.5f}'.format(
                datetime.now().time(), epoch, loss_train / n_batch))
    return losses_train


def compute_accuracy(model, loader, device=None):
    model.eval()
    embedding_dim = model.embedding.weight.data.shape[1]

    dist = 0
    total = 0

    with torch.no_grad():
        for contexts, targets in loader:
            contexts = contexts.to(device=device)
            targets = targets.to(device=device)

            outputs = model(contexts)
            _, predicted = torch.max(outputs, dim=1)
            total += len(targets)
            dist += torch.dist(predicted.float(), targets.float()) / embedding_dim

    acc =  dist / total
    return acc

In [11]:
batch_sizes = [64, 128, 256]
embedding_dims = [10, 16]

hparams = [{
    'batch_size': bs,
    'embedding_dim': em
 } for bs in batch_sizes for em in embedding_dims]

print(f"We are testing {len(hparams)} different hyper parameters.")

We are testing 6 different hyper parameters.


In [12]:
def train_best_model():

    # ---------------- Train many models ----------------------- 
    models = []
    train_accs = []
    val_accs = []

    for param in hparams:
        print(f'Now training with parameters {param}')
        train_loader = DataLoader(data_train, batch_size=param['batch_size'], shuffle=True)
        val_loader   = DataLoader(data_val, batch_size=param['batch_size'], shuffle=True)

        torch.manual_seed(seed)
        embedding = nn.Embedding(VOCAB_SIZE, param['embedding_dim'])
        torch.manual_seed(seed)
        model = Word2Vec(embedding).to(device=device)

        optimizer = optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.CrossEntropyLoss(weight=word_weigts)
        n_epochs=5
        train(n_epochs, optimizer, model, loss_fn, train_loader)

        train_acc = compute_accuracy(model, train_loader, device)
        print(f'Train accuracy: {train_acc}')
        val_acc = compute_accuracy(model, val_loader, device)
        print(f'Val accuracy:   {val_acc}')
        
        models.append(model)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        print()

    # ---------------- Retrain the best performing model for longer on more data -----------------------   
    best_idx = val_accs.index(min(val_accs))
    best_model = models[best_idx]
    best_param = hparams[best_idx]
    
    print(f'The best model had these parameters: {best_param}.')
    train_val_loader = DataLoader(ConcatDataset([data_train, data_val]), batch_size=best_param['batch_size'], shuffle=True)
    embedding = nn.Embedding(VOCAB_SIZE, best_param['embedding_dim'])
    torch.manual_seed(seed)
    best_model = Word2Vec(embedding).to(device=device)

    optimizer = optim.Adam(best_model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss(weight=word_weigts)
    n_epochs=50
    losses_train = train(n_epochs, optimizer, best_model, loss_fn, train_val_loader)

    return best_model

In [36]:
# ----------------------- Best Model -------------------------------
if os.path.isfile(PATH_GENERATED + 'best_model.pt'):
    best_model = torch.load(PATH_GENERATED + 'best_model.pt', map_location=torch.device(device))
else:
    best_model = train_best_model()
    torch.save(best_model, PATH_GENERATED + 'best_model.pt')

# ----------------------- Evaluate Best Model -------------------------------
test_loader = DataLoader(data_test, shuffle=True)
test_acc = compute_accuracy(best_model, test_loader)
print(test_acc)
print(compute_accuracy(best_model, DataLoader(data_train, shuffle=True)))
print(compute_accuracy(best_model, DataLoader(data_val, shuffle=True)))

tensor(59.4573)
tensor(60.5469)
tensor(60.7820)


In [85]:
# ----------------------- Calculate Cosine Similarity Matrix -------------------------------
cosineSimilarity = nn.CosineSimilarity(dim=2)
embedding = best_model.embedding
embedding_data = best_model.embedding.weight.data
cos_matrix = cosineSimilarity(embedding_data.unsqueeze(0), embedding_data.unsqueeze(1))

In [15]:
# ----------------------- Randomly select 10 words from 1000 most frequent -------------------------------
random.seed(seed)
selected_words = random.sample([word for word in vocab.lookup_tokens(range(100,VOCAB_SIZE)) if MAP_TARGET[vocab[word]] == 2], 10)
selected_indecies = vocab.lookup_indices(selected_words)
similar_words = []

for idx in selected_indecies:
    word_matrix = cos_matrix[idx].clone()
    word_matrix[idx] = -1 # Every word is most like itself 
    similar_words.append((vocab.lookup_token(torch.argmax(word_matrix)), torch.max(word_matrix)))

selected_similar_list = [(selected, similar, round(float(value), 2)) for (selected, (similar, value)) in zip(selected_words, similar_words)]
selected_similar_list

[('branch', 'proceed', 0.84),
 ('thrown', 'heap', 0.88),
 ('reach', 'towards', 0.76),
 ('provided', 'allowed', 0.75),
 ('week', 'play', 0.69),
 ('becomes', 'becoming', 0.76),
 ('motionless', 'erect', 0.77),
 ('repeated', 'hearing', 0.72),
 ('wounded', 'forgotten', 0.75),
 ('finally', 'abruptly', 0.79)]

In [32]:
# ----------------------- Convert embedding to tsv files -------------------------------
with open(PATH_GENERATED + 'vocab.tsv', 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
    for word in vocab.lookup_tokens(range(VOCAB_SIZE)):
        writer.writerow([word])
with open(PATH_GENERATED + 'embedding.tsv', 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    for word in embedding_data:
        word = [float(w) for w in word]
        writer.writerow(word)

# Conjunction of _be_ and _have_

In [80]:
con_list = ['be', 'am', 'are', 'is', 'was', 'were', 'been', 'being', 'have', 'has', 'had', 'having']

def compute_conjunction_label(w):
    return w in ['be', 'am', 'are', 'is', 'was', 'were', 'been', 'being', 'have', 'has', 'had', 'having']

CONJ_TARGET = {vocab[w]:compute_conjunction_label(w) for w in vocab.lookup_tokens(range(VOCAB_SIZE))}

def create_conjunction_dataset(text, vocab, context_size=CONTEXT_SIZE, map_target=CONJ_TARGET):
    
    # Transform each word to its index in the vocabulary.
    txt = [vocab[w] for w in text]

    n_text = len(text)
    contexts = []
    targets = []
    for i in range(n_text - context_size):
        
        t = txt[i + context_size]
        if map_target[t] == False: continue # We only want to guess conjunctions of be and have.
        c = txt[i:i + context_size]
        
        targets.append(t) 
        contexts.append(torch.tensor(c).to(device=device))
            
    contexts = torch.stack(contexts)
    targets = torch.tensor(targets).to(device=device)
    return TensorDataset(contexts, targets)

In [88]:
def load_conjunction_dataset(words, vocab, fname):
    """
    Load dataset if already generated, otherwise, create it and save it
    """
    
    if os.path.isfile(PATH_GENERATED + fname):
        dataset = torch.load(PATH_GENERATED + fname, map_location=torch.device(device))
    else:
        dataset = create_conjunction_dataset(words, vocab)
        torch.save(dataset, PATH_GENERATED + fname)
    return dataset

data_train_conj = load_conjunction_dataset(words_train, vocab, "conj_data_train.pt")
data_val_conj   = load_conjunction_dataset(words_val, vocab, "conj_data_val.pt")
data_test_conj  = load_conjunction_dataset(words_test, vocab, "conj_data_test.pt")

In [48]:
class SimpleMLP(nn.Module):
    def __init__(self, embedding, context_size=CONTEXT_SIZE):
        super().__init__()

        (vocab_size, embedding_dim) = embedding.weight.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.load_state_dict(embedding.state_dict())
        for p in self.embedding.parameters():
            p.requires_grad = False

        self.fc1 = nn.Linear(embedding_dim*context_size, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 12)

    def forward(self, x):
        out = self.embedding(x)
        out = F.relu(self.fc1(torch.flatten(out, 1)))
        out = F.relu(self.fc2(out, 1))
        out = self.fc3(out)

        return out

In [49]:
# ---------------- Simple MLP hyper parameters -----------------------
lrs = [0.01, 0.001]
decays = [0.8, 0.5, 0.1]
mlp_hparams = [{
    'lr': lr,
    'weight_decay': decay
} for lr in lrs for decay in decays]

In [89]:
train_losses = []
models = []
accuracies = []

for param in mlp_hparams:
    model = SimpleMLP(embedding)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), **param)
    train_loader = DataLoader(data_test_conj, batch_size=128, shuffle=True)
    val_loader = DataLoader(data_val_conj, batch_size=128, shuffle=True)
    n_epochs = 5
    
    loss = train(n_epochs, optimizer, model, loss_fn, train_loader)
    accuracy = compute_accuracy(model, val_loader)

    models.append(model)
    train_losses.append(loss)
    accuracies.append(accuracy)

TypeError: Linear.forward() takes 2 positional arguments but 3 were given